In [47]:
# Autoreload 
%load_ext autoreload
%autoreload 2

from pathlib import Path 
import pandas as pd 
import numpy as np
import os 
import re 
import gzip 
import shutil
import Bio.PDB.MMCIF2Dict
from typing import Union, List, Tuple, Dict, Optional
from pathlib import Path

pd.options.mode.chained_assignment = None  # default='warn'


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Get background frequencies.

In [ ]:
# TODO: 
# run compseq from within python.

In [85]:
# Load in file 
from phosphosite import DATA_DIR
from phosphosite.utils.compseq import parse_compseq_output, get_frequency_pair_dict, get_triplet_dict

filepath = DATA_DIR / "compseq" / "scgr4_human.composition" 
df = parse_compseq_output(filepath)

In [86]:
df

word,AAA,AAC,AAD,AAE,AAF,AAG,AAH,AAI,AAK,AAL,...,YYP,YYQ,YYR,YYS,YYT,YYU,YYV,YYW,YYY,Other
observations,,,,,,,,,,,,,,,,,,,,,
count,13401.000000,1535.000000,2641.000000,4821.000000,2529.000000,6470.000000,1628.000000,2534.000000,3158.000000,7917.000000,...,469.000000,501.000000,719.000000,895.000000,514.000000,0.0,546.000000,164.000000,478.000000,72.000000
freq,0.001179,0.000135,0.000232,0.000424,0.000223,0.000570,0.000143,0.000223,0.000278,0.000697,...,0.000041,0.000044,0.000063,0.000079,0.000045,0.0,0.000048,0.000014,0.000042,0.000006
exp,0.000345,0.000113,0.000233,0.000349,0.000179,0.000323,0.000129,0.000213,0.000282,0.000490,...,0.000045,0.000034,0.000040,0.000059,0.000038,0.0,0.000042,0.000009,0.000019,0.000006
freq/exp,3.421967,1.193803,0.998603,1.214486,1.240463,1.761144,1.110940,1.047225,0.986669,1.422139,...,0.922630,1.305517,1.583844,1.333848,1.193247,0.0,1.137856,1.677797,2.230995,1.094863


In [87]:
total_count = df.sum(axis=1)
total_count

observations
count       1.136161e+07
freq        1.000005e+00
exp         9.999825e-01
freq/exp    2.899393e+05
dtype: float64

In [88]:
t1 = "AAA"
t2 = "ACA"

df[[t1,t2]].sum(axis=1).freq

0.0012931

In [89]:
psite_dict=get_triplet_dict(df, metric="freq", amino_acids=None, centre_residue="STY")
len(psite_dict)

1323

In [90]:
std_residues = "ACDEFGHIKLMNPQRSTVWY"
psite_dict=get_triplet_dict(df, metric="freq", amino_acids=std_residues, centre_residue="STY")
len(psite_dict), 20*3*20

(1200, 1200)